# Proyecto películas

## Vamos a obtener todos los títulos, links y posiciones

Para ello hacemos un scrapeo con selenium para poder llegar al final de la página y clicar en el botón para que carguen más películas que las 30 primeras de la página principal ya que queremos un base de datos con un mayor número de películas.

In [5]:
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup


# Instalamos y obtenemos la ubicación del controlador
chromedriver_autoinstaller.install()

# Configuramos las opciones del navegador
chrome_options = Options()
chrome_options.headless = False

# Inicializamos el controlador con las opciones
driver = webdriver.Chrome(options=chrome_options)

# Abrimos la página web
driver.get('https://www.filmaffinity.com/es/topgen.php?genres=&chv=0&orderby=avg&movietype=full%7C&country=&fromyear=1874&toyear=2024&ratingcount=2&runtimemin=0&runtimemax=7')
driver.maximize_window()

# Definimos el límite deseado
limite_scrolls = 50 #scrolls verticales

# Inicializamos el contador de scrolls
contador_scrolls = 0 

# Realiza un bucle haciendo clic en el botón para cargar más contenido
while True:
    try:
        # Espera hasta que el botón sea visible
        load_more_button = WebDriverWait(driver, 2).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="load-more-bt"]/i'))
        )

        # Utiliza ActionChains para hacer clic en el botón con JavaScript
        ActionChains(driver).move_to_element(load_more_button).click().perform()

        # Espera un breve tiempo para que se cargue el contenido dinámico (ajusta según sea necesario)
        WebDriverWait(driver, 3).until(lambda driver: driver.execute_script('return document.readyState') == 'complete')

        # Actualiza el contador de películas
        scrolls = driver.find_elements(By.XPATH, '//*[@id="top-movies"]/li[1]/ul/li[2]/div')
        contador_scrolls += len(scrolls)

        # Sale del bucle si se alcanza o supera el límite
        if contador_scrolls >= limite_scrolls:
            break
        else:
            print(f"Contador de scrolls: {contador_scrolls}, Scrolls realizados  en esta iteración: {len(scrolls)}")

    except NoSuchElementException as e:
        print(f"Elemento no encontrado: {e}")
        continue
    except Exception as e:
        print(f"Error al hacer clic en el botón: {e}")
        break

# Obtiene el contenido de la página después de cargar todas las películas
page_source = driver.page_source

# Puedes continuar con BeautifulSoup para extraer información adicional
# Ejemplo:
soup = BeautifulSoup(page_source, 'html.parser')
# Aquí puedes usamos soup para extrarer películas y links a través del contenedor y unos bucles

contenedor_iterar = soup.find_all("div", attrs = {"class" : "mc-title"})

películas = []

for película in contenedor_iterar:
    películas.append(película.find("a")["title"].strip())

links = []

for link in contenedor_iterar:
    links.append(link.find("a")["href"])

# Aquí usamos soup para extraer la posición de cada película

contenedor_pos = soup.find_all("li", attrs = {"class" : "position"})

posición = []
for i in contenedor_pos:
    posición.append(i.text)
    
    
# Cerrar el navegador al finalizar
driver.quit()

Contador de scrolls: 1, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 2, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 3, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 4, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 5, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 6, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 7, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 8, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 9, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 10, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 11, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 12, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 13, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 14, Scrolls realizados  en esta iteración: 1
Contador de scrolls: 15, Scrolls realizados  en esta iteración: 1
Contador de scrolls

In [6]:
len(películas)

900

In [7]:
len(links)

900

In [8]:
import pandas as pd

# Creamos un diccionario
data = {
    'Películas': películas,
    'Links': links
}

# Crear el DataFrame
df_películas = pd.DataFrame(data)

# Mostrar el DataFrame
df_películas

,Películas,Links
0,El padrino,https://www.filmaffinity.com/es/film809297.html
1,Planeta Tierra II,https://www.filmaffinity.com/es/film107937.html
2,El padrino. Parte II,https://www.filmaffinity.com/es/film730528.html
3,The Wire (Bajo escucha),https://www.filmaffinity.com/es/film399474.html
4,Breaking Bad,https://www.filmaffinity.com/es/film489970.html
...,...,...
895,Marcado por el odio,https://www.filmaffinity.com/es/film822329.html
896,Ben-Hur,https://www.filmaffinity.com/es/film694490.html
897,Surcos,https://www.filmaffinity.com/es/film323900.html
898,Una mujer bajo la influencia,https://www.filmaffinity.com/es/film135032.html


## Vamos ahora a obtener el resto de información que queremos de cada link

#### Información que queremos:

- Título
-  año
- titulo original
- duracion
- País
- duracion
- director
- guión
- reparto
- música
- fotografía
- compañías
- género
- sinopsis
- premios
- puntuación
- número de votaciones

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [10]:
url = "https://www.filmaffinity.com/es/film809297.html"
response = requests.get(url)
bool(response)
soup = BeautifulSoup(response.text, "html.parser")

In [11]:
# Si en los que tuve que hacer find_all no vale para la función quizás tenga que hacer contenedores para todos, lo veremos, de momento vamos así.
# Parece que todos siguen la misma estrucutra así que podría servir

In [12]:
# Vamos a ver el título
título = soup.find("h1").text.strip()
título

'El padrino'

In [13]:
# Vamos a ver el título_original
título_original = soup.find("dd").text.strip()
título_original

'The Godfather'

In [14]:
# Año 
año = soup.find("dd", attrs = {"itemprop" : "datePublished"}).text
año

'1972'

In [15]:
# Duración
duración = soup.find("dd", attrs = {"itemprop" : "duration"}).text
duración
#Luego querremos quitar los min

'175 min.'

In [16]:
# País
país = soup.find("img", class_="nflag")["alt"]
país

'Estados Unidos'

In [17]:
# Dirección, segunda opción por si cambia en otras webs:

dirección = soup.find("span", attrs={"class": "nb"}).find("a").text
dirección

'Francis Ford Coppola'

In [18]:
# Posición, viene de lo anterior, aquí no viene

In [19]:
# Guión
contenedor_guión = soup.find_all("div", attrs = {"class" : "credits"})
contenedor_guión

[<div class="credits"><span class="nb" itemprop="director" itemscope="" itemtype="http://schema.org/Person"><a class="link" href="https://www.filmaffinity.com/es/name.php?name-id=845159519" itemprop="url" title="Francis Ford Coppola"><span itemprop="name">Francis Ford Coppola</span></a></span></div>,
 <div class="credits"><span class="nb"><a href="https://www.filmaffinity.com/es/name.php?name-id=845159519" title="Francis Ford Coppola">Francis Ford Coppola</a>,</span> <span class="nb"><a href="https://www.filmaffinity.com/es/name.php?name-id=745455954" title="Mario Puzo">Mario Puzo</a>. </span><i> Novela:</i> <span class="nb"><a href="https://www.filmaffinity.com/es/name.php?name-id=745455954" title="Mario Puzo">Mario Puzo</a></span></div>,
 <div class="credits"><span class="nb"><a href="https://www.filmaffinity.com/es/name.php?name-id=873781462" title="Nino Rota">Nino Rota</a></span></div>,
 <div class="credits"><span class="nb"><a href="https://www.filmaffinity.com/es/name.php?name-id

In [25]:
# Convertir cada nombre a un formato que incluye comillas
reparto = ['"{0}"'.format(nombre) for nombre in reparto]

# Convertir la lista a una cadena de texto con comas
reparto = ', '.join(reparto).replace('"', '')
reparto

''

In [26]:
# Encontrar el elemento <dt> que contiene la etiqueta "Música"
dt_musica = soup.find("dt", string="Música")

# Verificar si se encontró el elemento <dt>
if dt_musica:
    # Encontrar el siguiente elemento <dd> que contiene la información sobre la música
    contenedor_musica = dt_musica.find_next("div", class_="credits")
    
    # Verificar si se encontró el elemento <dd>
    if contenedor_musica:
        # Extraer el texto dentro de <dd>
        música = contenedor_musica.text.strip()
       
    else:
        print("No se encontró la información de música.")
else:
    print("No se encontró la etiqueta 'Música'.")


In [27]:
música

'Nino Rota'

In [28]:
# Fotografía
# Encontrar el elemento <dt> que contiene la etiqueta "Música"
dt_foto = soup.find("dt", string="Fotografía")

# Verificar si se encontró el elemento <dt>
if dt_musica:
    # Encontrar el siguiente elemento <dd> que contiene la información sobre la música
    contenedor_foto= dt_foto.find_next("div", class_="credits")
    
    # Verificar si se encontró el elemento <dd>
    if contenedor_foto:
        # Extraer el texto dentro de <dd>
        fotografía = contenedor_foto.text.strip()
    else:
        print("No se encontró la información de música.")
else:
    print("No se encontró la etiqueta 'Música'.")

In [29]:
fotografía

'Gordon Willis'

In [30]:
# Compañías
# Encontrar el elemento <dt> que contiene la etiqueta "Música"
dt_compañias = soup.find("dt", string="Compañías") 

# Verificar si se encontró el elemento <dt>
if dt_compañias:
    # Encontrar el siguiente elemento <dd> que contiene la información sobre la música
    contenedor_compañias = dt_compañias.find_next("div", class_="credits")
    
    # Verificar si se encontró el elemento <dd>
    if contenedor_compañias:
        # Extraer el texto dentro de <dd>
        compañías = contenedor_compañias.text.strip()
    else:
        print("No se encontró la información de música.")
else:
    print("No se encontró la etiqueta 'Música'.")


In [31]:
compañías

'Paramount Pictures, Alfran Productions.  Productor: Albert S. Ruddy'

In [32]:
# Género/ Categoría
contenedor_género = soup.find("dd", attrs = {"class" : "card-genres"})
contenedor_género

<dd class="card-genres">
<span itemprop="genre"><a href="https://www.filmaffinity.com/es/moviegenre.php?genre=DR&amp;attr=rat_count&amp;nodoc">Drama</a></span> |                 <a href="https://www.filmaffinity.com/es/movietopic.php?topic=959297&amp;attr=rat_count&amp;nodoc">Mafia</a>.                 <a href="https://www.filmaffinity.com/es/movietopic.php?topic=524594&amp;attr=rat_count&amp;nodoc">Crimen</a>.                 <a href="https://www.filmaffinity.com/es/movietopic.php?topic=503186&amp;attr=rat_count&amp;nodoc">Años 40</a>.                 <a href="https://www.filmaffinity.com/es/movietopic.php?topic=306617&amp;attr=rat_count&amp;nodoc">Años 50</a>.                 <a href="https://www.filmaffinity.com/es/movietopic.php?topic=279137&amp;attr=rat_count&amp;nodoc">Familia</a>.                 <a href="https://www.filmaffinity.com/es/movietopic.php?topic=436447&amp;attr=rat_count&amp;nodoc">Película de culto</a> </dd>

In [35]:
géneros = contenedor_género.text.strip()
géneros

'Drama |                 Mafia.                 Crimen.                 Años 40.                 Años 50.                 Familia.                 Película de culto'

In [36]:
# Sinopsis
sinopsis = soup.find("dd", attrs = {"itemprop":"description"}).text
sinopsis

"América, años 40. Don Vito Corleone (Marlon Brando) es el respetado y temido jefe de una de las cinco familias de la mafia de Nueva York. Tiene cuatro hijos: Connie (Talia Shire), el impulsivo Sonny (James Caan), el pusilánime Fredo (John Cazale) y Michael (Al Pacino), que no quiere saber nada de los negocios de su padre. Cuando Corleone, en contra de los consejos de 'Il consigliere' Tom Hagen (Robert Duvall), se niega a participar en el negocio de las drogas, el jefe de otra banda ordena su asesinato. Empieza entonces una violenta y cruenta guerra entre las familias mafiosas. (FILMAFFINITY)"

In [37]:
# Premios, los quitamos porque son una lista.
contenedor_premios = soup.find("dl", atttrs = {"class":"margin-top movie-info"})
contenedor_premios

In [38]:
# Encontrar el elemento <dd> con la clase "award"
contenedor_premios = soup.find("dd", class_="award")

# Iterar sobre los elementos <div> dentro de <dd> que contienen los enlaces <a>
premios = []
for div in contenedor_premios.find_all('div', class_='margin-bottom'):
    # Imprimir el texto dentro de cada div
    premios.append(div.text.strip())
premios

['1972: 3 Oscars: Mejor película, Actor (Marlon Brando), Guión adaptado. 11[10*] nom.',
 '1972: 5 Globos de Oro: Película (Drama), Director, Actor (Brando), Guión y BSO',
 '1972: Premios BAFTA: Mejor música. 5 nominaciones, incluyendo Mejor actor (Brando)',
 '1972: Círculo de Críticos de Nueva York: Mejor actor secundario (Duvall). 4 nominaciones',
 '1972: National Board of Review: Mejor actor sec. (Pacino) y Mejores 10 films del año',
 '1972: Sindicato de Directores (DGA): Mejor director']

In [39]:
# Convertir cada elemento a un formato que incluye comillas
premios = ['"{0}"'.format(premio) for premio in premios]

# Convertir la lista a una cadena de texto con comas y luego eliminar las comillas
premios = ', '.join(premios).replace('"', '')
premios

'1972: 3 Oscars: Mejor película, Actor (Marlon Brando), Guión adaptado. 11[10*] nom., 1972: 5 Globos de Oro: Película (Drama), Director, Actor (Brando), Guión y BSO, 1972: Premios BAFTA: Mejor música. 5 nominaciones, incluyendo Mejor actor (Brando), 1972: Círculo de Críticos de Nueva York: Mejor actor secundario (Duvall). 4 nominaciones, 1972: National Board of Review: Mejor actor sec. (Pacino) y Mejores 10 films del año, 1972: Sindicato de Directores (DGA): Mejor director'

In [40]:
# Puntuación
puntuación = soup.find("div", attrs = {"itemprop" : "ratingValue"}).text.strip()
puntuación

'9,0'

In [41]:
# Número de votos
votos = soup.find("span", attrs = {"itemprop" : "ratingCount"}).text.strip()
votos

'175.294'

 Título
-  año
- titulo original
- duracion
- País
- duracion
- director
- guión
- reparto
- música
- fotografía
- compañçias
- género
- sinopsis
- premios
- puntuación
- número de votaciones

In [42]:
import pandas as pd

# Crear un diccionario con las listas
data = {
    'Título': título,
    'Año': año,
    'Duración' : duración, 
    'País' : país,
    'Dirección' : dirección,
    'Música' : música, 
    'Fotografía' : fotografía,
    'Compañías' : compañías,
    'Sinopsis': sinopsis,
    'Puntuación' : puntuación,
    'Número de votaciones' : votos,   
}

# Crear el DataFrame
df_películas_indiv = pd.DataFrame(data, index = [0])

# Mostrar el DataFrame
df_películas_indiv

,Título,Año,Duración,País,Dirección,Música,Fotografía,Compañías,Sinopsis,Puntuación,Número de votaciones
0,El padrino,1972,175 min.,Estados Unidos,Francis Ford Coppola,Nino Rota,Gordon Willis,"Paramount Pictures, Alfran Productions. Produ...","América, años 40. Don Vito Corleone (Marlon Br...","9,0",175.294


# Creamos una función para automatizar

In [46]:
def pelis(url):
    response = requests.get(url)
    bool(response)
    soup = BeautifulSoup(response.text, "html.parser")
    título = soup.find("h1").text.strip()
    año = soup.find("dd", attrs = {"itemprop" : "datePublished"}).text
    duración = soup.find("dd", attrs = {"itemprop" : "duration"}).text
    país = soup.find("img", class_="nflag")["alt"]
    dirección = soup.find("span", attrs={"class": "nb"}).find("a").text
    #Falta reparto aquí
    dt_musica = soup.find("dt", string="Música")
    if dt_musica:
        contenedor_musica = dt_musica.find_next("div", class_="credits")
        if contenedor_musica:
            música = contenedor_musica.text.strip()
    dt_foto = soup.find("dt", string="Fotografía")
    if dt_foto:
        contenedor_foto= dt_foto.find_next("div", class_="credits")
        if contenedor_foto:
            fotografía = contenedor_foto.text.strip()
            
    dt_compañias = soup.find("dt", string="Compañías") 
    if dt_compañias:
        contenedor_compañias = dt_compañias.find_next("div", class_="credits")
        if contenedor_compañias:
            compañías = contenedor_compañias.text.strip()
    contenedor_premios = soup.find("dl", attrs = {"class":"margin-top movie-info"})
    premios = []
    for div in contenedor_premios.find_all('div', class_='margin-bottom'):
        premios.append(div.text.strip())
    premios = ['"{0}"'.format(premio) for premio in premios]
    premios = ', '.join(premios).replace('"', '')
    sinopsis = soup.find("dd", attrs = {"itemprop":"description"}).text
    puntuación = soup.find("div", attrs = {"itemprop" : "ratingValue"}).text.strip()
    votos = soup.find("span", attrs = {"itemprop" : "ratingCount"}).text.strip()
    return [título, año, duración, país, dirección, música, fotografía, compañías, premios, sinopsis, puntuación, votos]
    #Falta reparto en el return
    
   

# Recorremos todos los links con la función automatizada

In [47]:
urls = df_películas["Links"].values #cogemos los valores de la columnas links 

In [48]:
# Lista para almacenar los resultados
resultados = []

# Iterar sobre los enlaces y aplicar la función pelis
for index, row in df_películas.iterrows():
    try:
        print(f'Procesando URL {index + 1}/{len(df_películas)}: {row["Links"]}')
        resultados.append(pelis(row["Links"]))
    except Exception as e:
        print(f'Error en la URL {row["Links"]}: {e}')
        continue

# Crear un nuevo DataFrame con los resultados
columns = ['Título', 'Año', 'Duración', 'País', 'Dirección', 'Música', 'Fotografía', 'Compañías', 'Premios', 'Sinopsis', 'Puntuación', 'Votos']
df_resultados = pd.DataFrame(resultados, columns=columns)

# Concatenar los DataFrames original y de resultados
df_final = pd.concat([df_películas, df_resultados], axis=1)

# Mostrar el DataFrame final
print(df_final)

Procesando URL 1/900: https://www.filmaffinity.com/es/film809297.html
Procesando URL 2/900: https://www.filmaffinity.com/es/film107937.html
Procesando URL 3/900: https://www.filmaffinity.com/es/film730528.html
Procesando URL 4/900: https://www.filmaffinity.com/es/film399474.html
Procesando URL 5/900: https://www.filmaffinity.com/es/film489970.html
Procesando URL 6/900: https://www.filmaffinity.com/es/film518489.html
Error en la URL https://www.filmaffinity.com/es/film518489.html: cannot access local variable 'fotografía' where it is not associated with a value
Procesando URL 7/900: https://www.filmaffinity.com/es/film601451.html
Procesando URL 8/900: https://www.filmaffinity.com/es/film695552.html
Procesando URL 9/900: https://www.filmaffinity.com/es/film661074.html
Procesando URL 10/900: https://www.filmaffinity.com/es/film656153.html
Procesando URL 11/900: https://www.filmaffinity.com/es/film667376.html
Procesando URL 12/900: https://www.filmaffinity.com/es/film356260.html
Procesando

In [49]:
df_final

,Películas,Links,Título,Año,Duración,País,Dirección,Música,Fotografía,Compañías,Premios,Sinopsis,Puntuación,Votos
0,El padrino,https://www.filmaffinity.com/es/film809297.html,El padrino,1972,175 min.,Estados Unidos,Francis Ford Coppola,Nino Rota,Gordon Willis,"Paramount Pictures, Alfran Productions. Produ...",,"América, años 40. Don Vito Corleone (Marlon Br...","9,0",175.294
1,Planeta Tierra II,https://www.filmaffinity.com/es/film107937.html,Planeta Tierra II (Miniserie de TV),2016,50 min.,Estados Unidos,Elizabeth White,"Hans Zimmer, Jasha Klebe, Jacob Shea","Paul Stewart, Richard Wollocombe, Mateo Willis...","BBC, ZDF, France Télévisions, BBC Natural Hist...",,Miniserie de TV (6 episodios). La segunda part...,"8,9",4.389
2,El padrino. Parte II,https://www.filmaffinity.com/es/film730528.html,El padrino. Parte II,1974,200 min.,Estados Unidos,Francis Ford Coppola,"Nino Rota, Carmine Coppola",Gordon Willis,Coppola Co. Production. Productor: Francis Fo...,,Continuación de la historia de los Corleone po...,"8,9",140.009
3,The Wire (Bajo escucha),https://www.filmaffinity.com/es/film399474.html,The Wire (Bajo escucha) (Serie de TV),2002,60 min.,Estados Unidos,David Simon,Varios,Uta Briesewitz,Distribuidora: HBO,,Serie de TV (2002-2008). 5 temporadas. 60 epi...,"8,8",49.332
4,Breaking Bad,https://www.filmaffinity.com/es/film489970.html,Breaking Bad (Serie de TV),2008,45 min.,Estados Unidos,Vince Gilligan,Dave Porter,"Michael Slovis, Reynaldo Villalobos, Nelson Cr...","Gran Via Productions, High Bridge Productions,...",,Serie de TV (2008-2013). 5 temporadas. 62 epis...,"8,8",104.080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,Marcado por el odio,https://www.filmaffinity.com/es/film822329.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896,Ben-Hur,https://www.filmaffinity.com/es/film694490.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
897,Surcos,https://www.filmaffinity.com/es/film323900.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898,Una mujer bajo la influencia,https://www.filmaffinity.com/es/film135032.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# Guardamos en un archivo CSV
df_final.to_csv('películas_scrapeo.csv', index=False)